In [88]:
!pip install emoji

    100% |████████████████████████████████| 51kB 2.6MB/s 
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [103]:
import pandas as pd
import numpy as np
import emoji
%cd ../../data/chatbot
!ls

/usr/local/bin/notebooks/data/chatbot
Consumer_Complaints.csv		 frames-data-v1.tar.gz	   twcs.csv
cl2017sample.tar.gz		 frames.json		   twcs_tmobile.csv
customer-support-on-twitter.zip  telco-customer-churn.zip


In [14]:
df = pd.read_csv('twcs_tmobile.csv')
df.head()
# df.dtypes

,Unnamed: 0,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,620,1066,TMobileHelp,False,Tue Oct 31 22:28:46 +0000 2017,"@115910 Hey, Trevor! Send us a DM, and we'll t...",NaN,1067.0
1,627,1156,TMobileHelp,False,Tue Oct 31 22:24:01 +0000 2017,@115912 @115913 Thanks for getting that DM sen...,NaN,1157.0
2,630,1160,TMobileHelp,False,Tue Oct 31 22:23:40 +0000 2017,@115914 Yay for T-Mobile Tuesdays- lets's get ...,NaN,1161.0
3,631,1161,115914,True,Tue Oct 31 22:19:01 +0000 2017,@TMobileHelp trying to redeem a free tuesday c...,1160,NaN
4,632,1162,TMobileHelp,False,Tue Oct 31 22:22:53 +0000 2017,"@115915 Well, we'd be happy to take a look at ...",NaN,1163.0


In [95]:
df_cust = df[df.author_id!='TMobileHelp']
len(df_cust)
# df_cust.text[:5]

21849

In [7]:
df_tm = df[df.author_id=='TMobileHelp']
len(df_tm)

34317

In [81]:
import re,string

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
#     print(text)
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
#         print(word)
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    if words:
        return ' '.join(words)


# tests = [
# #     "@peter I really love that shirt at #Macy. http://bet.ly//WjdiW4",
# #     "@shawn Titanic tragedy could have been prevented Economic Times: Telegraph.co.ukTitanic tragedy could have been preve... http://bet.ly/tuN2wx",
# #     "I am at Starbucks http://4sh.com/samqUI (7419 3rd ave, at 75th, Brooklyn)",
#     '@TMobileHelp trying to redeem a free tuesday code and its not letting me telling me theres an error can i get help with this?'
# ]
# for t in tests:
#     print(strip_all_entities(strip_links(t)))


In [90]:
def give_emoji_free_text(text):
    allchars = [str for str in text.decode('utf-8')]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.decode('utf-8').split() if not any(i in str for i in emoji_list)])
    return clean_text

In [166]:
df = df_cust.copy()#.head(100)
# df = df_tm.copy()#.head(100)
df.text = df.text.apply(lambda x:give_emoji_free_text(str(x).encode('utf8')))
df.text = df.text.apply(lambda x:strip_all_entities(strip_links(x)))
set(df.text)

df = df.sort_values(by='author_id')
print(len(df.text))

df = df[df.in_response_to_tweet_id.isna()]
print(len(df.text))

df.text = df.text.replace(to_replace='None', value=np.nan).dropna()
print(len(df.text))
df.head()

21849
7710
7710


,Unnamed: 0,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
51949,2597213,2768576,115892,True,Tue Nov 21 11:34:54 +0000 2017,Hi Referred my friend to TMo They told her in ...,2768575,NaN
40513,1950265,2105295,115892,True,Wed Nov 08 12:59:46 +0000 2017,hi I understand there were some issues with th...,2105294,NaN
33683,1728988,1882847,115892,True,Mon Oct 23 01:18:07 +0000 2017,my Tmobile digits app been working fine for a ...,1882846,NaN
3,631,1161,115914,True,Tue Oct 31 22:19:01 +0000 2017,trying to redeem a free tuesday code and its n...,1160,NaN
18168,1057704,1170486,115916,True,Tue Oct 24 19:20:34 +0000 2017,is a little difficult when there is no data si...,1170485,NaN


In [186]:
txt_upgrade_phone = df[(df.text.str.contains('upgrade', na=False)) & (df.text.str.contains('phone|iphone|iPhone', na=False))]
txt_upgrade_phone = txt_upgrade_phone[~(txt_upgrade_phone.text.str.contains('contract|account|hi|Hi|HI', na=False))]
print(len(txt_upgrade_phone))
txt_upgrade_phone.head()

80


,Unnamed: 0,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
18964,1102819,1219486,117235,True,Wed Oct 25 14:41:56 +0000 2017,are there any deals to upgrade to the iphoneX ...,1219484,NaN
543,35676,45437,126055,True,Sun Nov 26 00:46:41 +0000 2017,I just upgraded my phone at verizon how long d...,45436,NaN
28783,1542750,1692865,128365,True,Mon Nov 06 17:30:36 +0000 2017,I have a question about the iPhone X upgrade o...,1692864,NaN
1802,137931,167094,140111,True,Fri Nov 24 20:25:39 +0000 2017,it seems like UPS lost my iPhone upgrade and e...,167093,NaN
1486,114807,142361,148134,True,Fri Nov 24 12:30:55 +0000 2017,No BF promotion for existing customers who wan...,142356,NaN


In [201]:
txt_acct = df[(df.text.str.contains('account|contract|balance|charges|monthly|total|query', na=False))]
# txt_acct = txt_acct[~(txt_acct.text.str.contains('contract', na=False))]
txt_acct = txt_acct[~(txt_acct.text.str.contains('hi|Hi|HI|hello|good', na=False))]
print(len(txt_acct))
txt_acct.head()

345


,Unnamed: 0,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
28065,1490656,1636779,117838,True,Sun Nov 05 21:22:20 +0000 2017,does a prepaid balance still restart the 90 da...,1636778,NaN
8718,551694,620760,119569,True,Wed Nov 22 17:13:04 +0000 2017,I just received my new iPhone 10 before I use ...,620759,NaN
474,29873,38349,124342,True,Wed Nov 01 14:37:11 +0000 2017,switcher over last month tried to set up voice...,38348,NaN
41740,2024773,2182895,125460,True,Thu Nov 09 16:02:17 +0000 2017,i have been trying to return my JOD but it see...,2182894,NaN
51553,2570034,2741054,128365,True,Mon Nov 20 17:53:31 +0000 2017,I need help managing phone lines on my account...,2741053,NaN


In [198]:
# txt_contract = df[(df.text.str.contains('contract', na=False))]
# txt_contract = txt_contract[~(txt_contract.text.str.contains('account', na=False))]
# txt_contract = txt_contract[~(txt_contract.text.str.contains('hi|Hi|HI', na=False))]
# print(len(txt_contract))
# txt_contract.head()

In [187]:
intents_upgrade = pd.DataFrame()
intents_upgrade['expression'] = txt_upgrade_phone.text
intents_upgrade['language'] = 'en'
intents_upgrade.head()
intents_upgrade.to_csv('intents_upgrade.csv', sep=';', index=False)

In [200]:
intents_acct = pd.DataFrame()
intents_acct['expression'] = txt_acct.text
intents_acct['language'] = 'en'
intents_acct.head()
intents_acct.to_csv('intents_acct.csv', sep=';', index=False)

In [197]:
# intents_contract = pd.DataFrame()
# intents_contract['expression'] = txt_contract.text
# intents_contract['language'] = 'en'
# intents_contract.head()
# intents_contract.to_csv('intents_contract.csv', sep=';', index=False)